In [2]:
import numpy as np
import trimesh
import scipy

In [6]:
# create test and train set 
# save it as txt files
import glob
import random
def get_random_train_list():
    files = glob.glob('data/xyz/*.xyz')
    train = random.sample(files, 120)
    train_set = set(train)
    train_list = []
    test_list = []
    for file in files:
        name = file.split('/')[-1].split('.')[0]
        if file in train_set:
            train_list.append(name)
        else:
            test_list.append(name)

    return train_list, test_list
def write_list_to_file(filename,my_list):
    with open(filename, 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)
train_list, test_list = get_random_train_list()
write_list_to_file('data/trainingset.txt', train_list)
write_list_to_file('data/testset.txt', test_list)

In [95]:
from random import sample
import glob
import trimesh
def save_xyz(pts, file_name):
    # print(pts)
    s = trimesh.util.array_to_string(pts)
    with open(file_name, 'w') as f:
        f.write("%s\n" % s)
def sample_k(matrix,k):
    # matrix = matrix.toarray()
    result = []
    for i,row in enumerate(matrix):
        row = list(row)
        if len(row) >= k:
            k_nearest = sample(row,k)
        else:
            while len(row) < k:
                row.append(0.0)
            k_nearest = row
        result.append(k_nearest)
    return result
def save_vertices():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        vertices = mesh.vertices
        dest_name = 'data/xyz/' +file.split('/')[-1].split('.')[0] +'.xyz'
        save_xyz(vertices, dest_name)
        
def create_target():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        result = knn_cotangent_laplacian(mesh, 6)
    
        dest_name = 'data/laplacian/' +file.split('/')[-1].split('.')[0] +'.laplacian'
        save_xyz(result, dest_name)
        break

create_target()


data/noisy/meshes_march2_mesh_0120.obj


In [93]:
from scipy.sparse import coo_matrix, eye
from scipy.spatial import distance
def get_cotangent_weight(i,neighbor,neighbors,norms, KNN):
    def get_k_h(i,j):
        list1, list2 = neighbors[i], neighbors[j]
        #if there is morethan 2 intersection choose the first 2 points
        intersection = list(set(list1).intersection(list2))
        try:
            k,h = intersection
        except:
            k,h = intersection[:2]
            # print(intersection)
            # print(i,j)
            # print('k,h', )

        return k,h
    def get_distance(i,j,k,h):
        j_idx = neighbors[i].index(j)
        lij = norms[i][j_idx]

        k_idx = neighbors[j].index(k)
        ljk = norms[j][k_idx]

        i_idx = neighbors[k].index(i)
        lki = norms[k][i_idx]

        h_idx = neighbors[j].index(h)
        ljh = norms[j][h_idx]

        i_idx = neighbors[h].index(i)
        lhi = norms[h][i_idx]

        return lij, ljk, lki, ljh, lhi 

    result = []
    for j in KNN:
        #if j in neighbor find Cotangent weight else assign 0
        wij = 0
        if j in neighbor:
            
            k,h  = get_k_h(i,j)
            lij, ljk, lki, ljh, lhi = get_distance(i,j,k,h)

            s_ijk = (lij + ljk + lki)/2
            A_ijk = 8 *  np.sqrt(s_ijk * ( s_ijk - lij) * ( s_ijk- ljk) * ( s_ijk - lki))

            s_ijh = (lij + ljh + lhi)/2
            A_ijh = 8 *  np.sqrt(s_ijh * ( s_ijh - lij) * ( s_ijh- ljh) * ( s_ijh - lhi))

            
            wij = ((-lij**2 + ljk ** 2 + lki**2)/  A_ijk)  + ((-lij**2 + ljh ** 2 + lhi**2)/  A_ijh)
        
       
        result.append(wij)
        
    return result

def knn_cotangent_laplacian(mesh, k):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]



    # norms = [i / i.sum() for i in norms]
    data = []

    for i, KNN in enumerate(closest):
        neighbor = neighbors[i]
        weight = get_cotangent_weight(i,neighbor, neighbors,norms, KNN)
        data.append(weight)
        # create the sparse matrix
    
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])
    
   

    data = np.concatenate([i / np.array(i).sum() if np.array(i).sum()>0 else i for i in data])
    
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix


In [90]:

def make_smooth(mesh, matrix= None):
    if matrix != None:
        matrix = matrix
    else:
        print("Matrix = None")
        matrix = trimesh.smoothing.laplacian_calculation(mesh, equal_weight= True)
        
    smoothed_mesh = trimesh.smoothing.filter_taubin(mesh,
                    lamb=0.4,
                    nu=0.5,
                    iterations=10,
                    laplacian_operator=matrix)
    return smoothed_mesh
smooth_path = 'data/smooth/meshes_squat2_mesh_0230.obj'
mesh_path = 'data/noisy/meshes_squat2_mesh_0230.obj'
outpath = 'output/meshes_squat2_mesh_0230_ctg_6.obj'
mesh = trimesh.load_mesh(mesh_path)
matrix_cotangent = knn_cotangent_laplacian(mesh,6)
smoothed =  make_smooth(mesh,matrix_cotangent)
output = smoothed.export(outpath)
smoothed_mesh = trimesh.load_mesh(outpath)
smoothed_mesh.show()
# print('=============================\n', matrix)


In [52]:
mesh1, mesh2 = 'output/meshes_bouncing_mesh_0020_smooth_ctg.obj', 'output/meshes_bouncing_mesh_0020_smooth.obj'
calculate_loss(mesh2, smooth_path)
#(tensor(0.0778), tensor(0.1726))
#(tensor(0.0915), tensor(0.1714))
# (tensor(0.0392), tensor(0.1713))

#(tensor(0.0394), tensor(0.1700)) ctg, smooth
# (tensor(0.0948), tensor(0.1706)) normal, smooth

loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0948)
MSE:  tensor(0.1706)


(tensor(0.0948), tensor(0.1706))

In [39]:
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
from chamferdist import ChamferDistance
def calculate_loss(mesh1, mesh2):
    loss = ChamferDistance()
    mse_loss = torch.nn.MSELoss()
    mesh1 = trimesh.load_mesh(mesh1)
    mesh2 = trimesh.load_mesh(mesh2)
    v1, v2 = mesh1.vertices.view(np.ndarray),mesh2.vertices.view(np.ndarray)
    v1, v2  = torch.tensor(v1).float().unsqueeze(0), torch.tensor(v2).float().unsqueeze(0)
    
    chamfer_loss = loss(v1, v2)
    mse_loss = mse_loss(v1, v2)
    print('===================================')
    print('loss(smooth_gt,noisy_gt)')
    print('ChamferDistance: ', chamfer_loss)
    print('MSE: ', mse_loss)
    print('===================================')
    return chamfer_loss, mse_loss

mesh1 = 'data/smooth/meshes_bouncing_mesh_0020.obj'
mesh2 = 'data/noisy/meshes_bouncing_mesh_0020.obj'

chamfer, mse = calculate_loss(outpath, mesh1)
# print('===================================')
# print('loss(smooth_gt,noisy_gt)')
# print('ChamferDistance: ', chamfer)
# print('MSE: ', mse)
# print('===================================')
'''
===================================
between Equal weight False & True: loss(smooth_gt,smooth_true)
ChamferDistance:  tensor(0.0306)
MSE:  tensor(0.1742)

===================================

loss(smooth_gt,smooth_pred_true)
ChamferDistance:  tensor(0.0948)
MSE:  tensor(0.1706)
===================================

===================================
loss(smooth_gt,smooth_pred_false)
ChamferDistance:  tensor(0.0399)
MSE:  tensor(0.1719)
===================================

===================================
loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0296)
MSE:  tensor(0.1707)
===================================
'''

loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0394)
MSE:  tensor(0.1700)


'\n===================================\nbetween Equal weight False & True: loss(smooth_gt,smooth_true)\nChamferDistance:  tensor(0.0306)\nMSE:  tensor(0.1742)\n\n===================================\n\nloss(smooth_gt,smooth_pred_true)\nChamferDistance:  tensor(0.0948)\nMSE:  tensor(0.1706)\n===================================\n\n===================================\nloss(smooth_gt,smooth_pred_false)\nChamferDistance:  tensor(0.0399)\nMSE:  tensor(0.1719)\n===================================\n\n===================================\nloss(smooth_gt,noisy_gt)\nChamferDistance:  tensor(0.0296)\nMSE:  tensor(0.1707)\n===================================\n'